In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import pandas as pd
import os, glob
import pathlib

Define Functions

Import Data Function

In [2]:
def IGA_Import(file):
    df_header = pd.read_csv(
        file,
        sep='= ',
        nrows=33, 
        header=None
    )
    
    df = pd.read_csv(
        file,
        delim_whitespace=True, 
        skiprows=35, 
        skipfooter=1, 
        header=None, 
        engine='python'
    )
    
    header = df_header[1].values.tolist()
    df.columns = header
    
    return(df)

In [49]:
def HumidityCheck(Adsorption):
    # Antoine Equation Paramters
    A = 8.07131
    B = 1730.63
    C = 233.426

    #P/P0 for water set by the MFC (torr)
    P_0 = 10**(A-(B/(C+Adsorption['VG1 Temp (°C)'])))
    P_w = 10**(A-(B/(C+Adsorption['VG1 Temp (°C)'])))*(
        Adsorption['Inlet A NITROGEN (ml/min)']/
        (Adsorption['Inlet A NITROGEN (ml/min)']+
         Adsorption['Inlet B NITROGEN (ml/min)']+
         Adsorption['Inlet D NITROGEN (ml/min)'])) 

    RH = (P_w.iat[-1]/P_0.iat[-1])*100   
    return(RH)

In [4]:
def CarbonCapture(Adsorption, Desorption):
    Adsorbed_Mass_Uptake = (
        (Adsorption['Total Weight (mg)'].max()-Adsorption['Total Weight (mg)'].min())/
        (Adsorption['Total Weight (mg)'].min())*100)
    
    p_H2O = 18.016*((Desorption['H2O (torr)'].fillna(0)/Desorption['N2 (torr)'].fillna(0)).sum())
    p_CO2 = 44.01*((Desorption['CO2 (torr)'].fillna(0)/Desorption['N2 (torr)'].fillna(0)).sum())
    
    CO2_Fraction = p_CO2/(p_H2O+p_CO2)
    CapturedCarbon = CO2_Fraction*Adsorbed_Mass_Uptake
    
    print('CO2 Fraction:', CO2_Fraction)
    print('Carbon Captured:', CapturedCarbon, 'wt.% CO2')
    
    return(Adsorbed_Mass_Uptake,CapturedCarbon)

Import Data

In [43]:
data1 = IGA_Import(file='Data_WO2F4-1-Ni_75%RH_400ppm_293K_300sccm_2h-Adsorption.txt')
activation = data1.iloc[:7195]
adsorption = data1.iloc[15602:22792]
data2 = IGA_Import(file='Data_WO2F4-1-Ni_75%RH_400ppm_293K_300sccm_2h-Desorption.txt')
desorption = data2.iloc[:7198]

In [133]:
data3 = IGA_Import(file='Data_NbOFFIVE-1-Ni_75%RH_400ppmCO2_293K_300sccm_2h-Adsorption.txt')
Nb_ads = data3.iloc[:6934]
Nb_des = data3.iloc[6935:13854]
data4 = IGA_Import(file='Data_NbOFFIVE-1-Ni_75%RH_400ppmCO2_293K_300sccm_2h-Desorption.txt')
Nb_activation = data4.iloc[:7097]

Some Calculations

Capture CO2 Calculation Function

In [52]:
Adsorbed_Mass_Uptake, CarbonCaptured = CarbonCapture(Adsorption=adsorption,Desorption=desorption)
print("total uptake:", Adsorbed_Mass_Uptake, 'wt. %')
RelativeHumidity = HumidityCheck(Adsorption=adsorption)
print("Relative Humidity:", RelativeHumidity, '%')

CO2 Fraction: 0.2929032668053835
Carbon Captured: 3.013020816149681 wt.% CO2
total uptake: 10.286743637282992 wt. %
Relative Humidity: 74.99082630016348 %
